import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/kaggle/input/usa-housing-price/USA_Housing.csv')

In [ ]:
df.head()

In [ ]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [ ]:
df.info()

# Model Train test split

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, 
                                       random_state=42)

In [ ]:
print(len(train_set), len(test_set))

# Feature Engineering

## Imputation using sklearn.impute

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
df_num = df.select_dtypes(include=[np.number])

In [ ]:
df_num.head()

In [ ]:
df_num.median().values

In [ ]:
imputer.fit(df_num)

In [ ]:
X = imputer.transform(df_num)

In [ ]:
X

In [ ]:
df_tr = pd.DataFrame(X, columns=df_num.columns,
                          index=df_num.index)

In [ ]:
df_tr.head()

# Outlier Detection using Isolation forest

In [ ]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(X)

In [ ]:
outlier_pred

In [ ]:
df_tr['outlier_pred'] = outlier_pred

In [ ]:
df_tr.head()

In [ ]:
df_tr.loc[df_tr['outlier_pred']==-1]

# Handling Text and Categorical Attributes

In [ ]:
df.info()

In [ ]:
df['Address']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
# Male = 0 , female =1
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(df)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(df)

## Feature Scaling

In [ ]:
df_tr.head()

In [ ]:
df_tr = df_tr.drop('outlier_pred',axis = 1)

In [ ]:
df_tr.head()

In [ ]:
df_scale = df_tr.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
df_num_min_max_scaled = min_max_scaler.fit_transform(df_scale)

In [ ]:
df_num_min_max_scaled

In [ ]:
df_scale = df_num.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
df_num_std_scaled = std_scaler.fit_transform(df_scale)

In [ ]:
len(df_num_std_scaled)

# Apply simple linear Regression

In [ ]:
df_std_scaled = pd.DataFrame(df_num_std_scaled, columns=df_num.columns,
                          index=df_num.index)

In [ ]:
df_std_scaled.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df_std_scaled, test_size=0.2, 
                                       random_state=42)

In [ ]:
train_set.info()

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(train_set[["Avg. Area Income","Avg. Area House Age"]],train_set['Price'])



In [ ]:
scaled_predictions = model.predict(test_set[["Avg. Area Income","Avg. Area House Age"]])


In [ ]:
from sklearn.metrics import r2_score

r2_score(test_set['Price'], scaled_predictions)